In [ ]:
import pandas as pd
import os

# Notebookファイルからの相対パスでCSVファイルのパスを指定
csv_file_path = '../data/train.csv'

# ファイルの存在確認
if not os.path.exists(csv_file_path):
    print(f"エラー: ファイルが見つかりません。パスを確認してください: {os.path.abspath(csv_file_path)}")
else:
    try:
        # CSVファイルをDataFrameに読み込む
        # 'time'列 (0番目の列) をインデックスに指定し、同時に日付型としてパースする
        df_train = pd.read_csv(csv_file_path, index_col='time', parse_dates=['time'])
        # または index_col=0, parse_dates=[0] でも同じ結果になります。

        print("DataFrameの読み込みと'time'列のインデックス化・日付型変換が完了しました。")

        # 読み込んだDataFrameの最初の5行を表示して確認
        # print("\nDataFrameの先頭5行:")
        # print(df_train.head())

        # DataFrameのインデックス情報を確認
        # print("\nDataFrameのインデックス情報:")
        # print(df_train.index)

        # DataFrameの基本情報を表示 (time列がインデックスになっていることを確認)
        # print("\nDataFrameの情報:")
        # df_train.info()

    except Exception as e:
        print(f"エラー: CSVファイルの読み込み中に問題が発生しました: {e}")

# この時点で df_train は 'time' 列を DatetimeIndex として持っています。
# この後の欠損値処理などは、この df_train に対して行います。

In [ ]:
pd.set_option('display.max_columns', None)
df_train.head()

In [ ]:
df_train.info()

In [ ]:
df_train.describe()

In [ ]:
# --- 欠損値補完 ---
# 以前の分類に基づく連続変数のリスト
continuous_cols = [
    'generation_biomass', 'generation_fossil_brown_coal/lignite',
    'generation_fossil_gas', 'generation_fossil_hard_coal', 'generation_fossil_oil',
    'generation_hydro_pumped_storage_consumption', 'generation_hydro_run_of_river_and_poundage',
    'generation_hydro_water_reservoir', 'generation_nuclear', 'generation_other',
    'generation_other_renewable', 'generation_solar', 'generation_waste',
    'generation_wind_onshore', 'total_load_actual'
]

# 1. 0で補完するのが適切なカラム
cols_to_fill_zero = [col for col in df_train.columns if 'rain' in col or 'snow' in col]
cols_to_fill_zero.append('generation_solar')

print(f"0で補完するカラム: {cols_to_fill_zero}")
for col in cols_to_fill_zero:
    if col in df_train.columns:
        df_train[col] = df_train[col].fillna(0)

# 2. 残りの連続変数に対して前方補完
# continuous_colsは以前定義した連続変数のリスト
remaining_continuous_cols = [col for col in continuous_cols if col not in cols_to_fill_zero]

print("前方補完（ffill）を実行するカラム（一部抜粋）:", remaining_continuous_cols[:5])
for col in remaining_continuous_cols:
    if col in df_train.columns:
        df_train[col] = df_train[col].fillna(method='ffill')

# 3. それでも先頭に残った欠損を後方補完（bfill）で埋める
# （データの先頭から欠損が続いている場合、ffillでは埋まらないため）
df_train = df_train.fillna(method='bfill')

print("\n欠損値の補完が完了しました。")
# 補完後に欠損がないか確認
print("補完後の欠損値の数:\n", df_train.isnull().sum())

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

# このコードを実行する前に、df_trainがロードされている必要があります。
# 例: df_train = pd.read_csv('your_data.csv', index_col='time', parse_dates=True)

# --- ステップ①: 風向をサイン・コサインに変換 ---
print("ステップ①: 風向の処理を開始します...")
cities = ['valencia', 'madrid', 'bilbao', 'barcelona', 'seville']
original_wind_deg_cols = []

for city in cities:
    wind_deg_col = f'{city}_wind_deg'
    original_wind_deg_cols.append(wind_deg_col)

    # 度数をラジアンに変換
    radians = np.deg2rad(df_train[wind_deg_col])

    # 新しいサイン・コサイン特徴量を作成
    df_train[f'{city}_wind_deg_cos'] = np.cos(radians)
    df_train[f'{city}_wind_deg_sin'] = np.sin(radians)

# 元の風向きカラムを削除
df_train = df_train.drop(columns=original_wind_deg_cols)
print(f"元の風向カラム {original_wind_deg_cols} を削除し、サイン・コサインカラムを10個追加しました。")
print("-" * 30)


# --- ステップ②: 連続変数の標準化 ---
print("ステップ②: 連続変数の標準化を開始します...")

# 以前の分類に基づく連続変数のリスト
# continuous_cols = [
#     'generation_biomass', 'generation_fossil_brown_coal/lignite',
#     'generation_fossil_gas', 'generation_fossil_hard_coal', 'generation_fossil_oil',
#     'generation_hydro_pumped_storage_consumption', 'generation_hydro_run_of_river_and_poundage',
#     'generation_hydro_water_reservoir', 'generation_nuclear', 'generation_other',
#     'generation_other_renewable', 'generation_solar', 'generation_waste',
#     'generation_wind_onshore', 'total_load_actual'
# ]
for city in cities:
    continuous_cols.extend([
        f'{city}_temp', f'{city}_temp_min', f'{city}_temp_max', f'{city}_pressure',
        f'{city}_humidity', f'{city}_wind_speed', f'{city}_rain_1h', f'{city}_rain_3h',
        f'{city}_snow_3h', f'{city}_clouds_all'
    ])

# ステップ①で追加したサイン・コサインカラムもリストに追加
for city in cities:
    continuous_cols.extend([f'{city}_wind_deg_cos', f'{city}_wind_deg_sin'])

# 欠損値があるとエラーになるため、対象カラムを絞り込み
# （実際には、標準化の前に欠損値処理を行うのが望ましいです）
cols_to_scale = [col for col in continuous_cols if col in df_train.columns and df_train[col].isnull().sum() == 0]
print(f"標準化の対象となるカラム数: {len(cols_to_scale)}")

# 標準化の実行
scaler = StandardScaler()
df_train[cols_to_scale] = scaler.fit_transform(df_train[cols_to_scale])

print("連続変数の標準化が完了しました。")
print("-" * 30)

# 処理後のデータフレームの先頭を表示して確認
print("処理後のデータフレームの先頭5行:")
print(df_train.head())

In [ ]:
# # --- 欠損値補完 ---

# # 1. 0で補完するのが適切なカラム
# cols_to_fill_zero = [col for col in df_train.columns if 'rain' in col or 'snow' in col]
# cols_to_fill_zero.append('generation_solar')

# print(f"0で補完するカラム: {cols_to_fill_zero}")
# for col in cols_to_fill_zero:
#     if col in df_train.columns:
#         df_train[col] = df_train[col].fillna(0)

# # 2. 残りの連続変数に対して前方補完
# # continuous_colsは以前定義した連続変数のリスト
# remaining_continuous_cols = [col for col in continuous_cols if col not in cols_to_fill_zero]

# print("前方補完（ffill）を実行するカラム（一部抜粋）:", remaining_continuous_cols[:5])
# for col in remaining_continuous_cols:
#     if col in df_train.columns:
#         df_train[col] = df_train[col].fillna(method='ffill')

# # 3. それでも先頭に残った欠損を後方補完（bfill）で埋める
# # （データの先頭から欠損が続いている場合、ffillでは埋まらないため）
# df_train = df_train.fillna(method='bfill')

# print("\n欠損値の補完が完了しました。")
# # 補完後に欠損がないか確認
# print("補完後の欠損値の数:\n", df_train.isnull().sum())

In [ ]:
import pandas as pd

# このコードを実行する前に、df_trainが前回の処理を終えた状態でロードされている必要があります。

print("処理前のデータフレームの形状:", df_train.shape)
print("処理前のカラム（一部）:", df_train.columns[:10].tolist(), "...")
print("-" * 30)

# --- ステップ①: weather_descriptionカラムを削除 ---
print("ステップ①: weather_descriptionカラムの削除を開始します...")
cities = ['valencia', 'madrid', 'bilbao', 'barcelona', 'seville']
description_cols_to_drop = [f'{city}_weather_description' for city in cities]

# 存在するカラムのみを削除対象とする
existing_cols_to_drop = [col for col in description_cols_to_drop if col in df_train.columns]
if existing_cols_to_drop:
    df_train = df_train.drop(columns=existing_cols_to_drop)
    print(f"削除されたカラム: {existing_cols_to_drop}")
else:
    print("削除対象のdescriptionカラムは見つかりませんでした。")

print("-" * 30)


# --- ステップ②: 残りのカテゴリ変数にOne-Hot Encodingを適用 ---
print("ステップ②: One-Hot Encodingを開始します...")

# One-Hot Encodingの対象となるカラムのリストを作成
categorical_cols_to_encode = []
for city in cities:
    categorical_cols_to_encode.extend([
        f'{city}_weather_id',
        f'{city}_weather_main',
        f'{city}_weather_icon'
    ])

# df_trainに存在するカラムのみを対象とする
existing_categorical_cols = [col for col in categorical_cols_to_encode if col in df_train.columns]

if existing_categorical_cols:
    # weather_idは数値型のため、カテゴリとして正しく扱われるように文字列に変換
    for col in existing_categorical_cols:
        if 'weather_id' in col:
            df_train[col] = df_train[col].astype(str)
    
    print(f"One-Hot Encoding対象のカラム: {existing_categorical_cols}")
    
    # One-Hot Encodingを実行
    # prefixを指定して、どの元のカラムから来た特徴量か分かりやすくします。
    # 元のカテゴリカルカラムは自動的に削除されます。
    df_train = pd.get_dummies(df_train, columns=existing_categorical_cols)
    
    print("One-Hot Encodingが完了しました。")
else:
    print("One-Hot Encoding対象のカラムは見つかりませんでした。")

print("-" * 30)
print("全ての処理が完了しました。")
print("最終的なデータフレームの形状:", df_train.shape)
print("処理後のカラム（一部）:", df_train.columns[:10].tolist(), "...")
print("\n処理後のデータフレームの先頭5行:")
print(df_train.head())

In [ ]:
df_train["price_actual"].describe()

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

# df_train が前処理済みの状態で存在すると仮定します。
# 例：df_train = pd.read_csv(...) などでロードされ、
# 欠損値補完、標準化、One-Hotエンコーディングが完了している状態。

In [ ]:
# def create_sequences(input_data, target_column, timesteps):
#     """
#     2Dの時系列データフレームから、LSTM用の3Dシーケンスデータを作成する関数
#     """
#     X_list, y_list = [], []
    
#     # 説明変数と目的変数に分離
#     # One-Hotエンコーディングにより目的変数の位置が変わっている可能性があるため、名前で指定
#     y = input_data[target_column].values
#     X_data = input_data.drop(columns=target_column).values
    
#     for i in range(len(X_data) - timesteps):
#         X_list.append(X_data[i:i+timesteps])
#         y_list.append(y[i+timesteps])
        
#     return np.array(X_list), np.array(y_list)

# # ハイパーパラメータ
# TIMESTEPS = 24
# TARGET_COLUMN = 'price_actual' # 目的変数のカラム名

# # ★★★ 修正箇所：ここから ★★★
# # bool型をint型に変換して、データ型を数値に統一します
# bool_cols = df_train.select_dtypes(include=['bool']).columns
# df_train[bool_cols] = df_train[bool_cols].astype(int)
# print("bool型をint型に変換しました。")
# # ★★★ 修正箇所：ここまで ★★★

# print("シーケンスデータを作成中...")
# X_seq, y_seq = create_sequences(df_train, TARGET_COLUMN, TIMESTEPS)

# print(f"X_seq shape: {X_seq.shape}") # -> (サンプル数, 24, 特徴量数)
# print(f"y_seq shape: {y_seq.shape}") # -> (サンプル数,)

In [ ]:
# # シーケンスを作成する前のdf_trainの状態を確認します。
# # このコードを、`create_sequences`関数を呼び出す前に追加してください。

# print("データフレームのデータ型情報を表示します:")
# df_train.info()

# print("\n" + "="*50 + "\n")

# # object型のカラムのみをリストアップ
# object_cols = df_train.select_dtypes(include=['object']).columns
# if not object_cols.empty:
#     print(f"エラーの原因となっている可能性のある'object'型のカラム: {object_cols.tolist()}")
# else:
#     print("'object'型のカラムは見つかりませんでした。他の原因が考えられます。")

In [ ]:
# # 訓練データと検証データの分割（例: 80%を訓練、20%を検証）
# train_size = int(len(X_seq) * 0.8)
# X_train, X_val = X_seq[:train_size], X_seq[train_size:]
# y_train, y_val = y_seq[:train_size], y_seq[train_size:]

# # PyTorchのテンソルに変換
# X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
# y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
# X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
# y_val_tensor = torch.tensor(y_val, dtype=torch.float32).unsqueeze(1)

# # DatasetとDataLoaderの作成
# BATCH_SIZE = 64 # バッチサイズも後で調整するハイパーパラメータ

# train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
# train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

# val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
# val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

# print("DataLoaderの準備が完了しました。")

In [ ]:
# class LSTMModel(nn.Module):
#     def __init__(self, input_size, hidden_size, num_layers, dropout_prob):
#         super(LSTMModel, self).__init__()
#         self.lstm = nn.LSTM(
#             input_size=input_size,
#             hidden_size=hidden_size,
#             num_layers=num_layers,
#             batch_first=True,
#             dropout=dropout_prob if num_layers > 1 else 0 # 1層の場合はdropoutは適用されない
#         )
#         self.dropout = nn.Dropout(dropout_prob)
#         self.fc = nn.Linear(hidden_size, 1)

#     def forward(self, x):
#         # LSTM層からの出力を取得
#         # h_n, c_n (最後の隠れ状態とセル状態) は今回は使用しない
#         lstm_out, _ = self.lstm(x)
        
#         # 最後のタイムステップの出力のみを選択
#         last_hidden_state = lstm_out[:, -1, :]
        
#         # ドロップアウト層
#         out = self.dropout(last_hidden_state)
        
#         # 全結合層（出力層）
#         out = self.fc(out)
#         return out

In [ ]:
# # ハイパーパラメータ（Optunaで探索する候補）
# INPUT_SIZE = X_train.shape[2] # 特徴量の数
# HIDDEN_SIZE = 100
# NUM_LAYERS = 2
# DROPOUT_PROB = 0.2
# LEARNING_RATE = 0.001
# NUM_EPOCHS = 20 # まずは少ないエポック数で試す

# # デバイスの設定
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# print(f"使用デバイス: {device}")

# # モデル、損失関数、オプティマイザのインスタンス化
# model = LSTMModel(INPUT_SIZE, HIDDEN_SIZE, NUM_LAYERS, DROPOUT_PROB).to(device)
# loss_fn = nn.MSELoss() # 平均二乗誤差
# optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

# print("モデルの学習を開始します...")
# for epoch in range(NUM_EPOCHS):
#     # --- 訓練モード ---
#     model.train()
#     total_train_loss = 0
#     for inputs, labels in train_loader:
#         inputs, labels = inputs.to(device), labels.to(device)
        
#         optimizer.zero_grad()
#         predictions = model(inputs)
#         loss = loss_fn(predictions, labels)
#         loss.backward()
#         optimizer.step()
        
#         total_train_loss += loss.item()
    
#     avg_train_loss = total_train_loss / len(train_loader)

#     # --- 検証モード ---
#     model.eval()
#     total_val_loss = 0
#     with torch.no_grad():
#         for inputs, labels in val_loader:
#             inputs, labels = inputs.to(device), labels.to(device)
#             predictions = model(inputs)
#             loss = loss_fn(predictions, labels)
#             total_val_loss += loss.item()
            
#     avg_val_loss = total_val_loss / len(val_loader)
    
#     print(f"Epoch [{epoch+1}/{NUM_EPOCHS}] | Train Loss: {avg_train_loss:.4f} | Validation Loss: {avg_val_loss:.4f}")

In [ ]:
import optuna
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import numpy as np

# --- このコードの前提 ---
# df_train: 全ての前処理が完了した訓練データ (DataFrame)
# LSTMModel: 定義済みのモデルクラス
# create_sequences: 定義済みのシーケンス作成関数
# TARGET_COLUMN = 'price_actual'
# device = 'cuda' or 'cpu'

# --- 1. Optunaの `objective` 関数を定義 ---
def objective(trial):
    # --- A. 探索対象のハイパーパラメータを定義 ---
    # trial.suggest_... で探索範囲を指定します
    timesteps = trial.suggest_categorical('timesteps', [24, 48, 72])
    hidden_size = trial.suggest_int('hidden_size', 32, 128, log=True)
    num_layers = trial.suggest_int('num_layers', 1, 3)
    dropout_prob = trial.suggest_float('dropout_prob', 0.1, 0.5)
    learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-2, log=True)
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128])

    # --- B. データ準備 (trialごとにtimestepsが変わるため、毎回作成) ---
    X_seq, y_seq = create_sequences(df_train, TARGET_COLUMN, timesteps)
    
    train_size = int(len(X_seq) * 0.8)
    X_train, X_val = X_seq[:train_size], X_seq[train_size:]
    y_train, y_val = y_seq[:train_size], y_seq[train_size:]

    X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
    X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
    y_val_tensor = torch.tensor(y_val, dtype=torch.float32).unsqueeze(1)

    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    
    # --- C. モデルと学習設定のインスタンス化 ---
    input_size = X_train.shape[2]
    model = LSTMModel(input_size, hidden_size, num_layers, dropout_prob).to(device)
    loss_fn = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    
    # --- D. モデルの学習と評価 ---
    # 最適化中はエポック数を少なめにして、高速に探索します
    NUM_EPOCHS_OPTIM = 10
    
    for epoch in range(NUM_EPOCHS_OPTIM):
        model.train()
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            predictions = model(inputs)
            loss = loss_fn(predictions, labels)
            loss.backward()
            optimizer.step()

    # 最後の検証ロスを評価値とする
    model.eval()
    total_val_loss = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            predictions = model(inputs)
            loss = loss_fn(predictions, labels)
            total_val_loss += loss.item()
            
    avg_val_loss = total_val_loss / len(val_loader)
    
    # --- E. 評価値を返す ---
    # Optunaはこの戻り値を最小化しようとします
    return avg_val_loss


# --- 2. Optunaの学習（最適化）を実行 ---
# `direction='minimize'`で、objective関数の戻り値（検証ロス）を最小化するよう指示
study = optuna.create_study(direction='minimize')

# n_trialsで試行回数を指定。まずは30-50回程度から試すのがおすすめです。
study.optimize(objective, n_trials=30)


# --- 3. 結果の確認 ---
print("\n最適化が完了しました。")
print("="*50)
print("最高のトライアル:")
best_trial = study.best_trial

print(f"  Value (最小検証ロス): {best_trial.value}")
print("  Params (最高のハイパーパラメータ): ")
for key, value in best_trial.params.items():
    print(f"    {key}: {value}")

In [ ]:
# 各試行の履歴をプロット
optuna.visualization.plot_optimization_history(study).show()

# 各ハイパーパラメータの重要度をプロット
optuna.visualization.plot_param_importances(study).show()

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import numpy as np

# --- このコードの前提 ---
# study: Optunaの最適化が完了したstudyオブジェクト
# df_train: 全ての前処理が完了した訓練データ (DataFrame)
# LSTMModel: 定義済みのモデルクラス
# create_sequences: 定義済みのシーケンス作成関数
# TARGET_COLUMN = 'price_actual'
# device = 'cuda' or 'cpu'

# --- 1. Optunaで見つけた最高のハイパーパラメータを取得 ---
best_params = study.best_params
print("Optunaによって見つかった最高のハイパーパラメータ:")
print(best_params)

# 各パラメータを変数に格納
FINAL_TIMESTEPS = best_params['timesteps']
FINAL_HIDDEN_SIZE = best_params['hidden_size']
FINAL_NUM_LAYERS = best_params['num_layers']
FINAL_DROPOUT_PROB = best_params['dropout_prob']
FINAL_LEARNING_RATE = best_params['learning_rate']
FINAL_BATCH_SIZE = best_params['batch_size']


# --- 2. 全ての訓練データを使用して最終的な学習データを準備 ---
print("\n最終モデル学習のため、全ての訓練データでシーケンスを作成中...")
X_final_train, y_final_train = create_sequences(df_train, TARGET_COLUMN, FINAL_TIMESTEPS)

X_final_train_tensor = torch.tensor(X_final_train, dtype=torch.float32)
y_final_train_tensor = torch.tensor(y_final_train, dtype=torch.float32).unsqueeze(1)

final_train_dataset = TensorDataset(X_final_train_tensor, y_final_train_tensor)
final_train_loader = DataLoader(final_train_dataset, batch_size=FINAL_BATCH_SIZE, shuffle=True)

print(f"最終的な学習用シーケンスの形状: {X_final_train_tensor.shape}")


# --- 3. 最終モデルの構築と学習の実行 ---
# 最高のパラメータでモデルを再構築
final_model = LSTMModel(
    input_size=X_final_train.shape[2],
    hidden_size=FINAL_HIDDEN_SIZE,
    num_layers=FINAL_NUM_LAYERS,
    dropout_prob=FINAL_DROPOUT_PROB
).to(device)

loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(final_model.parameters(), lr=FINAL_LEARNING_RATE)

FINAL_NUM_EPOCHS = 50 # 十分なエポック数で学習
print(f"\n最終モデルの学習を開始します... (Epochs: {FINAL_NUM_EPOCHS})")

final_model.train() # モデルを訓練モードに設定
for epoch in range(FINAL_NUM_EPOCHS):
    total_loss = 0
    for inputs, labels in final_train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        predictions = final_model(inputs)
        loss = loss_fn(predictions, labels)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    # 5エポックごとに進捗を表示
    if (epoch + 1) % 5 == 0:
        avg_loss = total_loss / len(final_train_loader)
        print(f"Epoch [{epoch+1}/{FINAL_NUM_EPOCHS}], Loss: {avg_loss:.4f}")


# --- 4. 学習済みモデルの保存 ---
model_save_path = 'final_lstm_model.pth'
torch.save(final_model.state_dict(), model_save_path)

print("\n学習が完了し、最終モデルを'{model_save_path}'として保存しました。")

In [ ]:
def create_sequences(input_data, target_column, timesteps):
    """
    2Dの時系列データフレームから、LSTM用の3Dシーケンスデータを作成する関数
    """
    X_list, y_list = [], []

    # 説明変数と目的変数に分離
    # One-Hotエンコーディングにより目的変数の位置が変わっている可能性があるため、名前で指定
    y = input_data[target_column].values
    X_data = input_data.drop(columns=target_column).values

    for i in range(len(X_data) - timesteps):
        X_list.append(X_data[i:i+timesteps])
        y_list.append(y[i+timesteps])

    return np.array(X_list), np.array(y_list)

# ハイパーパラメータ
TIMESTEPS = 48
TARGET_COLUMN = 'price_actual' # 目的変数のカラム名

# ★★★ 修正箇所：ここから ★★★
# bool型をint型に変換して、データ型を数値に統一します
bool_cols = df_train.select_dtypes(include=['bool']).columns
df_train[bool_cols] = df_train[bool_cols].astype(int)
print("bool型をint型に変換しました。")
# ★★★ 修正箇所：ここまで ★★★

print("シーケンスデータを作成中...")
X_seq, y_seq = create_sequences(df_train, TARGET_COLUMN, TIMESTEPS)

print(f"X_seq shape: {X_seq.shape}") # -> (サンプル数, 24, 特徴量数)
print(f"y_seq shape: {y_seq.shape}") # -> (サンプル数,)

In [ ]:
# シーケンスを作成する前のdf_trainの状態を確認します。
# このコードを、`create_sequences`関数を呼び出す前に追加してください。

print("データフレームのデータ型情報を表示します:")
df_train.info()

print("\n" + "="*50 + "\n")

# object型のカラムのみをリストアップ
object_cols = df_train.select_dtypes(include=['object']).columns
if not object_cols.empty:
    print(f"エラーの原因となっている可能性のある'object'型のカラム: {object_cols.tolist()}")
else:
    print("'object'型のカラムは見つかりませんでした。他の原因が考えられます。")

In [ ]:
# 訓練データと検証データの分割（例: 80%を訓練、20%を検証）
train_size = int(len(X_seq) * 0.8)
X_train, X_val = X_seq[:train_size], X_seq[train_size:]
y_train, y_val = y_seq[:train_size], y_seq[train_size:]

# PyTorchのテンソルに変換
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32).unsqueeze(1)

# DatasetとDataLoaderの作成
BATCH_SIZE = 32 # バッチサイズも後で調整するハイパーパラメータ

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

print("DataLoaderの準備が完了しました。")

In [ ]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, dropout_prob):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout_prob if num_layers > 1 else 0 # 1層の場合はdropoutは適用されない
        )
        self.dropout = nn.Dropout(dropout_prob)
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        # LSTM層からの出力を取得
        # h_n, c_n (最後の隠れ状態とセル状態) は今回は使用しない
        lstm_out, _ = self.lstm(x)
        
        # 最後のタイムステップの出力のみを選択
        last_hidden_state = lstm_out[:, -1, :]
        
        # ドロップアウト層
        out = self.dropout(last_hidden_state)
        
        # 全結合層（出力層）
        out = self.fc(out)
        return out

In [ ]:
# ハイパーパラメータ（Optunaで探索する候補）
INPUT_SIZE = X_train.shape[2] # 特徴量の数
HIDDEN_SIZE = 37
NUM_LAYERS = 2
DROPOUT_PROB = 0.2020769227005973
LEARNING_RATE = 0.0014057516838302802
NUM_EPOCHS = 50 # まずは少ないエポック数で試す

# デバイスの設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"使用デバイス: {device}")

# モデル、損失関数、オプティマイザのインスタンス化
model = LSTMModel(INPUT_SIZE, HIDDEN_SIZE, NUM_LAYERS, DROPOUT_PROB).to(device)
loss_fn = nn.MSELoss() # 平均二乗誤差
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

print("モデルの学習を開始します...")
for epoch in range(NUM_EPOCHS):
    # --- 訓練モード ---
    model.train()
    total_train_loss = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        predictions = model(inputs)
        loss = loss_fn(predictions, labels)
        loss.backward()
        optimizer.step()
        
        total_train_loss += loss.item()
    
    avg_train_loss = total_train_loss / len(train_loader)

    # --- 検証モード ---
    model.eval()
    total_val_loss = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            predictions = model(inputs)
            loss = loss_fn(predictions, labels)
            total_val_loss += loss.item()
            
    avg_val_loss = total_val_loss / len(val_loader)
    
    print(f"Epoch [{epoch+1}/{NUM_EPOCHS}] | Train Loss: {avg_train_loss:.4f} | Validation Loss: {avg_val_loss:.4f}")

In [ ]:
import pandas as pd
import os

# Notebookファイルからの相対パスでCSVファイルのパスを指定
csv_file_path = '../data/test.csv'

# ファイルの存在確認
if not os.path.exists(csv_file_path):
    print(f"エラー: ファイルが見つかりません。パスを確認してください: {os.path.abspath(csv_file_path)}")
else:
    try:
        # CSVファイルをDataFrameに読み込む
        # 'time'列 (0番目の列) をインデックスに指定し、同時に日付型としてパースする
        df_test_original = pd.read_csv(csv_file_path, index_col='time', parse_dates=['time'])
        # または index_col=0, parse_dates=[0] でも同じ結果になります。

        print("DataFrameの読み込みと'time'列のインデックス化・日付型変換が完了しました。")

        # 読み込んだDataFrameの最初の5行を表示して確認
        # print("\nDataFrameの先頭5行:")
        # print(df_test_original.head())

        # DataFrameのインデックス情報を確認
        # print("\nDataFrameのインデックス情報:")
        # print(df_test_original.index)

        # DataFrameの基本情報を表示 (time列がインデックスになっていることを確認)
        # print("\nDataFrameの情報:")
        # df_test_original.info()

    except Exception as e:
        print(f"エラー: CSVファイルの読み込み中に問題が発生しました: {e}")

# この時点で df_test_original は 'time' 列を DatetimeIndex として持っています。
# この後の欠損値処理などは、この df_test_original に対して行います。

In [ ]:
df_test_original.info()

In [ ]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import TensorDataset, DataLoader

# --- このコードの前提 ---
# df_train_original: 前処理前の元の訓練データ（'price_actual'を含む）
# df_test_original: 前処理前の元のテストデータ
# model: 学習済みのPyTorchモデル
# scaler: 訓練データの説明変数のみで学習済みのStandardScaler
# continuous_cols: 連続変数のカラム名リスト
# TIMESTEPS = 24
# BATCH_SIZE = 64
# device = 'cuda' or 'cpu'

# --- ステップ0: テストデータをクリーンな状態にする ---
# 元のテストデータをコピーして、他のセルの影響を排除する
df_test = df_test_original.copy()
print("テストデータの前処理を開始します...")
print(f"処理前のdf_testの形状: {df_test.shape}")

# --- ステップ1: 欠損値補完 ---
print("1. 欠損値を補完中...")
cols_to_fill_zero = [col for col in df_test.columns if 'rain' in col or 'snow' in col or 'solar' in col]
df_test[cols_to_fill_zero] = df_test[cols_to_fill_zero].fillna(0)
df_test = df_test.ffill().bfill() # FutureWarningを避けるため .ffill()/.bfill() に変更

# --- ステップ2: 風向の処理と不要カラムの削除 ---
print("2. 風向をサイン・コサインに変換し、不要なカラムを削除中...")
cities = ['valencia', 'madrid', 'bilbao', 'barcelona', 'seville']
original_wind_deg_cols = [f'{city}_wind_deg' for city in cities]
for col in original_wind_deg_cols:
    if col in df_test.columns:
        radians = np.deg2rad(df_test[col])
        df_test[f'{col}_cos'] = np.cos(radians)
        df_test[f'{col}_sin'] = np.sin(radians)
df_test = df_test.drop(columns=original_wind_deg_cols, errors='ignore')
description_cols_to_drop = [f'{city}_weather_description' for city in cities]
df_test = df_test.drop(columns=description_cols_to_drop, errors='ignore')

# --- ステップ3: 標準化 (訓練時のScalerを再利用) ---
print("3. 訓練時のScalerを再利用して標準化中...")
cols_to_scale_test = [col for col in continuous_cols if col in df_test.columns and col != 'price_actual']
df_test[cols_to_scale_test] = scaler.transform(df_test[cols_to_scale_test])

# --- ステップ4: One-Hot Encoding とカラムの整合 ---
print("4. One-Hot Encodingとカラムの整合性を確保中...")
categorical_cols_to_encode = []
for city in cities:
    categorical_cols_to_encode.extend([f'{city}_weather_id', f'{city}_weather_main', f'{city}_weather_icon'])
existing_categorical_cols = [col for col in categorical_cols_to_encode if col in df_test.columns]
for col in existing_categorical_cols:
    df_test[col] = df_test[col].astype(str)
df_test = pd.get_dummies(df_test, columns=existing_categorical_cols)

# 訓練データから目的変数を除いたカラムリストを基準にする
train_feature_columns = df_train.drop(columns=['price_actual']).columns
df_test = df_test.reindex(columns=train_feature_columns, fill_value=0)

bool_cols = df_test.select_dtypes(include=['bool']).columns
df_test[bool_cols] = df_test[bool_cols].astype(int)
print("前処理が完了しました。")

# --- これ以降は前回と同様の推論コード ---
# (中略：推論と提出ファイル作成のコードをここに続ける)

In [ ]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import TensorDataset, DataLoader

# --- このステップの前提 ---
# df_test: 全ての前処理が完了したテストデータ (DataFrame)
# model: 学習済みのPyTorchモデル
# device: 'cuda' or 'cpu'
# TIMESTEPS = 24 (訓練時と同じ値)

# --- ステップ1: テスト用シーケンスデータの作成 ---
# ターゲット（正解ラベル）がないため、説明変数Xのみのシーケンスを作成します
def create_test_sequences(input_data, timesteps):
    X_list = []
    X_data = input_data.values
    for i in range(len(X_data) - timesteps + 1):
        X_list.append(X_data[i:i+timesteps])
    return np.array(X_list)

print("テストデータから予測用のシーケンスを作成中...")
X_test = create_test_sequences(df_test, TIMESTEPS)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)

print(f"作成されたテストシーケンスの形状: {X_test.shape}")


# --- ステップ2: モデルによる予測の実行 ---
print("モデルによる予測を実行中...")

# テストデータ用のDataLoaderを作成（ラベルは不要）
test_dataset = TensorDataset(X_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False) # BATCH_SIZEは訓練時と同じ

# モデルを評価モードに切り替え
model.eval()

# 予測値を保存するリスト
all_predictions = []

# 勾配計算をオフにして予測
with torch.no_grad():
    for inputs in test_loader: # inputsはリストなので[0]でテンソルを取り出す
        inputs_tensor = inputs[0].to(device)
        predictions = model(inputs_tensor)
        all_predictions.append(predictions.cpu().numpy())

# リストを結合して一つのNumpy配列に
predictions_np = np.concatenate(all_predictions).flatten() # 1次元配列に変換

print("予測が完了しました。")


# # --- ステップ3: 提出用CSVファイルの作成 ---
# print("提出用CSVファイルを作成中...")

# # 予測値に対応するタイムスタンプを取得
# # create_test_sequencesでは (len - timesteps + 1) 個のシーケンスが作られる
# # しかし、提出形式はdf_testの全期間であるため、予測値の個数とタイムスタンプの個数を合わせる必要がある
# # ここでは、タイムスタンプの最後の部分に予測値を割り当てるのが一般的
# num_predictions = len(predictions_np)
# num_timestamps = len(df_test.index)

# # 提出用データフレームを作成
# submission_df = pd.DataFrame(index=df_test.index)
# submission_df['price_actual'] = np.nan # 一旦NaNで初期化

# # 最後の部分に予測値を割り当てる
# submission_df['price_actual'].iloc[-num_predictions:] = predictions_np

# # 予測できなかった先頭部分を直後の値で埋める（前方補完の逆）
# submission_df = submission_df.fillna(method='bfill')

# # 提出形式に合わせてCSVファイルとして保存
# # header=False とすることで、カラム名を入れずに出力します
# submission_df.to_csv('submission.csv', header=False)

# print("\n完了: 'submission.csv'という名前で提出ファイルが作成されました。")
# print("提出ファイルの内容（先頭5行）:")
# print(submission_df.head())

In [ ]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import TensorDataset, DataLoader

# --- このステップの前提 ---
# df_train: 全ての前処理が完了した訓練データ (DataFrame)
# df_test: 全ての前処理が完了したテストデータ (DataFrame)
# model: 学習済みのPyTorchモデル
# device: 'cuda' or 'cpu'
# TIMESTEPS = 24
# TARGET_COLUMN = 'price_actual'

# --- ステップ1: 訓練データの履歴をテストデータに結合 ---
print("1. 訓練データの末尾をテストデータの履歴として結合中...")

# ★★★ 修正箇所 ★★★
# df_trainから目的変数を除いた特徴量のみのスライスを取得
history = df_train.drop(columns=[TARGET_COLUMN]).iloc[-TIMESTEPS:]

# 助走期間とテストデータを結合
# これで history と df_test の両方が356カラムとなり、正しく結合されます
combined_test_data = pd.concat([history, df_test], ignore_index=False)

print(f"元のdf_testの形状: {df_test.shape}")
print(f"助走期間を追加したデータの形状: {combined_test_data.shape}")


# --- ステップ2: 結合データから予測用シーケンスを作成 ---
def create_test_sequences(input_data, timesteps):
    X_list = []
    X_data = input_data.values
    for i in range(len(X_data) - timesteps):
        X_list.append(X_data[i:i+timesteps])
    return np.array(X_list)

print("\n2. 予測用のシーケンスを作成中...")
X_test_with_history = create_test_sequences(combined_test_data, TIMESTEPS)
X_test_tensor = torch.tensor(X_test_with_history, dtype=torch.float32)

print(f"作成されたテストシーケンスの形状: {X_test_tensor.shape}")
assert len(X_test_tensor) == len(df_test)


# --- ステップ3: モデルによる予測の実行 ---
print("\n3. モデルによる予測を実行中...")

test_dataset = TensorDataset(X_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

model.eval()
all_predictions = []
with torch.no_grad():
    for inputs in test_loader:
        inputs_tensor = inputs[0].to(device)
        predictions = model(inputs_tensor)
        all_predictions.append(predictions.cpu().numpy())

predictions_np = np.concatenate(all_predictions).flatten()
print("予測が完了しました。")


# --- ステップ4: 提出用CSVファイルの作成 ---
print("\n4. 提出用CSVファイルを作成中...")

submission_df = pd.DataFrame(
    {'price_actual': predictions_np},
    index=df_test.index
)

submission_df.to_csv('../submissions/06_09_submission_01.csv', header=False)

print("\n完了: '06_09_submission_01.csv'という名前で最終的な提出ファイルが作成されました。")
print("提出ファイルの内容（先頭5行）:")
print(submission_df.head())